# Energy Storage - Battery Management

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedronahum/stochastic-optimization/blob/master/notebooks/energy_storage.ipynb)

In [ ]:
# Install JAX and dependencies
!pip install -q jax jaxlib jaxtyping chex numpy matplotlib

# Clone repository (force fresh clone for latest code)
import os
import shutil

if os.path.exists('stochastic-optimization'):
    shutil.rmtree('stochastic-optimization')

!git clone https://github.com/pedronahum/stochastic-optimization.git
os.chdir('stochastic-optimization')

# Clear Python import cache
import sys
for key in list(sys.modules.keys()):
    if key.startswith('problems'):
        del sys.modules[key]

print('✓ Setup complete!')

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from problems.energy_storage import EnergyStorageConfig, EnergyStorageModel

In [ ]:
# Create model configuration
config = EnergyStorageConfig(
    capacity=100.0,
    initial_energy=50.0,
    max_charge_rate=20.0,
    max_discharge_rate=20.0
)
model = EnergyStorageModel(config)
key = jax.random.PRNGKey(42)
state = model.init_state(key)
print('✓ Energy storage model ready')

In [ ]:
# Track battery and prices
battery_levels, prices, charges, rewards = [], [], [], []
for t in range(24):
    key, k1, k2 = jax.random.split(key, 3)
    decision = jnp.array([10.0])  # Charge/discharge amount
    exog = model.sample_exogenous(k2, state, t)
    reward = model.reward(state, decision, exog)
    battery_levels.append(float(state[0]))
    prices.append(float(exog.price) if hasattr(exog, 'price') else 1.0)
    charges.append(float(decision[0]))
    rewards.append(float(reward))
    state = model.transition(state, decision, exog)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(battery_levels)
plt.title('Battery Level')
plt.subplot(1, 3, 2)
plt.plot(prices)
plt.title('Electricity Price')
plt.subplot(1, 3, 3)
plt.plot(charges)
plt.axhline(0, color='red', linestyle='--')
plt.title('Charge/Discharge')
plt.tight_layout()
plt.show()
print(f'Total profit: {sum(rewards):.1f}')

## References

1. **Sequential Decision Analytics Lab** - https://castle.princeton.edu/sdalinks/
2. **Repository** - https://github.com/pedronahum/stochastic-optimization
3. **JAX Documentation** - https://jax.readthedocs.io/